### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2024-10-12 22:58:17--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1 [following]
--2024-10-12 22:58:18--  https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc376bec11e2922e9fd7f157bf9b.dl.dropboxusercontent.com/cd/0/inline/CcU6P7viN_2qMmepZfwk_d_aser4YUkQbiUGrftZSQnzmzmZ12qXRLsaLsiY96BOR2Pnuqi-T1DHvJRROXJzU6Kn5HJ_STBhC-6ZXqlFq28S3-3XoGP8LomNvMOtXI0fRhQ/file?dl=1# [following]
--2024-10-12 22:58:19--  https://uc376bec11e2922e9fd7f157bf9b.dl.dropbox

,author,day,id,link,month,summary,tag,title,year
2866,"[{'name': 'Mahmoud Hamandi'}, {'name': ""Mike D...",9,1803.03719v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,We present a novel human-aware navigation appr...,"[{'term': 'cs.RO', 'scheme': 'http://arxiv.org...",DeepMoTIon: Learning to Navigate Like Humans,2018
30290,"[{'name': 'Jhony K. Pontes'}, {'name': 'Chen K...",29,1711.10669v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,One challenge that remains open in 3D deep lea...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Image2Mesh: A Learning Framework for Single Im...,2017
21672,"[{'name': 'Ryan Prescott Adams'}, {'name': 'Da...",19,0710.3742v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,Changepoints are abrupt variations in the gene...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Bayesian Online Changepoint Detection,2007
13109,"[{'name': 'Adrian Šošić'}, {'name': 'Abdelhak ...",4,1605.01278v4,"[{'rel': 'related', 'href': 'http://dx.doi.org...",5,Learning from demonstration (LfD) is the proce...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",A Bayesian Approach to Policy Recognition and ...,2016
4016,"[{'name': 'Sandhya Arora'}, {'name': 'Debotosh...",21,1005.4032v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,In this paper we present an OCR for Handwritte...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Combining Multiple Feature Extraction Techniqu...,2010


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [4]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
tokenize = lambda paper: ' '.join(tokenizer.tokenize(paper.lower()))

lines = list(map(tokenize, lines))

In [5]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [6]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"


# def shift(t: tuple, n: int = 1):
#     """
#     Tuple (any iterable in fact) left circular shift
#     """
#     return t[n:] + t[:n]


def count_line_ngrams(counts: defaultdict, line: str, n: int) -> None:
    """
    Same as count_ngrams but for a single line.
    """
    context = (n - 1) * (UNK, )
    for token in (line.split() + [EOS]):
      counts[context][token] += 1
      context = (context + (token,))[1:]


def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
      count_line_ngrams(counts, line, n)

    return counts


In [7]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [8]:
def counter_mul(cnt : Counter, multiplier : float):
    """
    Counter defaultdict multiplication
    """
    prod = Counter()
    for key in cnt:
        prod[key] = cnt[key] * multiplier
    return prod


def counter_div(divident : Counter, divisor : float):
    """
    Counter defaultdict division
    """
    assert divisor != 0
    return counter_mul(divident, 1 / divisor)

def counter_pow(cnt : Counter, exp : float):
    """
    Counter defaultdict division
    """
    pow = Counter()
    for key in cnt:
        pow[key] = cnt[key] ** exp
    return pow


class NGramLanguageModel:
    """
    """
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for context in counts.keys():
            self.probs[context] = counter_div(counts[context], counts[context].total())


    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)


In [9]:
given = Counter({"a" : 10, "b" : 5, "c" : 3})

expected = Counter({"a" : 5, "b" : 2.5, "c" : 1.5})
assert counter_div(given, 2) == expected
assert counter_mul(expected, 2) == given

expected = Counter({"a" : 100, "b" : 25, "c" : 9})
assert counter_pow(given, 2.) == expected

Let's test it!

In [10]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [11]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [12]:
def counter_normalize(cnt : Counter, p: float):
    num = counter_pow(cnt, p)
    denum = num.total()
    return counter_div(num, denum)


In [13]:
def counter_sample(probs : Counter):
    assert abs(probs.total() - 1.) < 10e-6, f"probs sum into {probs.total()}"
    items = np.array(list(probs.keys()))
    probs = np.array(list(probs.values()))
    return np.random.choice(items, p=probs)


In [14]:
def counter_argmax(cnt : Counter):
    argmax, max = None, float('-inf')
    for key, value in cnt.items():
        if value < max:
            continue
        argmax, max = key, value

    return argmax


In [15]:
def get_next_token(lm: NGramLanguageModel, prefix: tuple, temperature: float=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    next_token_prob = lm.get_possible_next_tokens(prefix)
    if temperature == 0.:
        return counter_argmax(next_token_prob)
    else:
        sample_probs = counter_normalize(next_token_prob, 1 / temperature)
        return counter_sample(sample_probs)


In [16]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [17]:
prefix = 'machine' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

machine learning framework has been recent research treats it as a function over the last decade , we establish locally q - learning algorithms . our result is a function that involves 481 patients . _EOS_


In [18]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the two - stream convnets , which is a distributed algorithm , that is , the proposed system offers the capability of the proposed approach , we introduce a new model , and the results of the language . in this paper , we introduce a new approach for solving this problem , we show that , the proposed method , a new method for the first time , and that the proposed method is significantly faster than the other hand , the proposed method is proposed to solve this problem , we propose a new approach for


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [19]:
def num_tokens(lines : list[str]) -> int:
    """
    Counts num of tokens EOS included.
    """
    return len(' '.join(lines).split()) + len(lines)


In [20]:
def lm_log_prob(lm: NGramLanguageModel, line: str, min_logprob: float) -> float:
    prefix, log_prob = "", 0.

    for tok in (line.split() + [EOS]):
        if lm.get_next_token_prob(prefix, tok) == 0:
            log_prob += min_logprob
        else:
            log_prob += np.log(lm.get_next_token_prob(prefix, tok))
        prefix = ' '.join([prefix, tok])

    return log_prob


In [21]:
def perplexity(lm: NGramLanguageModel, lines: list[str], min_logprob: float = np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    this_lm_log_prob = lambda line: lm_log_prob(lm, line, min_logprob)
    log_prob = sum(map(this_lm_log_prob, lines))

    return np.exp((-1 / num_tokens(lines)) * log_prob)


In [22]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [23]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28543
N = 3, Perplexity = 61999196239911532363776.00000


In [24]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [25]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [26]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [27]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [28]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [29]:
from tqdm import tqdm

class KneserNeyLanguageModel(NGramLanguageModel):
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines: list[str], n: int, delta: float = 1.0):
        super().__init__(lines, n)
        self.delta = delta
        self.vocab = set(word for line in lines for word in line.split()) | {EOS}
        self.ordered_probs = self.calculate_probs(lines)

    def calculate_probs(self, lines: list[str]) -> dict:
        reverse_counts = defaultdict(int)
        total_pairs = 0

        # First order
        counts = count_ngrams(lines, 2)
        for prefix, local_counter in tqdm(counts.items(), leave=False, desc="Calculate first order", mininterval=0.5):
            for token in local_counter:
                reverse_counts[token] += 1
                total_pairs += 1

        ordered_probs = {1: {token: count / total_pairs for token, count in reverse_counts.items()}}
        ordered_probs[1][UNK] = self.delta / len(self.vocab)

        # Higher order
        for order in tqdm(range(2, self.n + 1), leave=False, desc="Calculate order", mininterval=0.5):
            ordered_probs[order] = {}
            counts = count_ngrams(lines, order)

            for prefix, local_counter in tqdm(counts.items(), leave=False, desc="Currently for prefix", mininterval=0.5):
                dict_for_prefix = defaultdict(int)
                local_sum = sum(local_counter.values())
                lambd = self.delta * len(local_counter) / local_sum

                for token in self.vocab:
                    count = local_counter[token] if token in local_counter else 0
                    dict_for_prefix[token] = max(0, count - self.delta) / local_sum

                    if order == 2:
                        dict_for_prefix[token] += lambd * ordered_probs[order - 1][token]
                    else:
                        dict_for_prefix[token] += lambd * ordered_probs[order - 1][prefix[1:]][token]

                ordered_probs[order][prefix] = dict_for_prefix

        return ordered_probs

    def get_possible_next_tokens(self, prefix: str) -> Counter:
        if self.n == 1:
            return self.ordered_probs[1]
        prefix_tokens = [tok if tok in self.vocab else UNK for tok in prefix.split()]
        prefix_tokens = prefix_tokens[max(0, len(prefix_tokens) - self.n + 1):]
        prefix_tokens = [UNK] * (self.n - 1 - len(prefix_tokens)) + prefix_tokens
        return self.probs[tuple(prefix_tokens)]

    def get_next_token_prob(self, prefix: str, next_token: str) -> float:
        if next_token not in self.vocab:
            next_token = UNK
        return self.get_possible_next_tokens(prefix)[next_token]


In [30]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

Currently for prefix:  88%|████████▊ | 1826/2086 [00:01<00:00, 1183.69it/s]


In [ ]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=0.5)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 1367.70568


Currently for prefix:   5%|▍         | 2593/54176 [03:51<1:20:21, 10.70it/s]